In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from scipy.stats import sem
#from pandas_profiling import ProfileReport

In [3]:
print(os.getcwd())

/Users/swang/Kaggle/AMP-PD-prediction/code


In [4]:
datapath="~/Kaggle/AMP-PD-prediction/data/"
protein_data = pd.read_csv(f"{datapath}train_proteins.csv")
target_data = pd.read_csv(f"{datapath}train_clinical_data.csv")
peptides_data = pd.read_csv(f"{datapath}train_peptides.csv")
protein_data.shape, target_data.shape, peptides_data.shape

((232741, 5), (2615, 8), (981834, 6))

In [21]:
print(f"""This dataset has {len(peptides_data.Peptide.unique())} peptides measured from {len(peptides_data.UniProt.unique())} proteins.
{len(peptides_data.patient_id.unique())} patients were measured {len(peptides_data.visit_month.unique())} times at month {peptides_data.visit_month.unique()}.""")


This dataset has 968 peptides measured from 227 proteins.
248 patients were measured 15 times at month [  0   3   6  12  18  24  30  36  48  54  60  72  84  96 108].


In [5]:
protein_data.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [6]:
protein_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232741 entries, 0 to 232740
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   visit_id     232741 non-null  object 
 1   visit_month  232741 non-null  int64  
 2   patient_id   232741 non-null  int64  
 3   UniProt      232741 non-null  object 
 4   NPX          232741 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 8.9+ MB


In [23]:
protein_data.isna().sum()

visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64

In [24]:
peptides_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981834 entries, 0 to 981833
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   visit_id          981834 non-null  object 
 1   visit_month       981834 non-null  int64  
 2   patient_id        981834 non-null  int64  
 3   UniProt           981834 non-null  object 
 4   Peptide           981834 non-null  object 
 5   PeptideAbundance  981834 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 44.9+ MB


In [25]:
peptides_data.isna().sum()

visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64

In [9]:
target_data.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [10]:
target_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615 entries, 0 to 2614
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   visit_id                             2615 non-null   object 
 1   patient_id                           2615 non-null   int64  
 2   visit_month                          2615 non-null   int64  
 3   updrs_1                              2614 non-null   float64
 4   updrs_2                              2613 non-null   float64
 5   updrs_3                              2590 non-null   float64
 6   updrs_4                              1577 non-null   float64
 7   upd23b_clinical_state_on_medication  1288 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 163.6+ KB


In [11]:
target_data.isna().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64

In [13]:
peptides_data.sort_values("PeptideAbundance")

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
126128,49683_0,0,49683,P02774,RTHLPEVFLSK,1.099850e+01
456992,4923_24,24,4923,P02774,RTHLPEVFLSK,1.970570e+01
566125,51846_24,24,51846,P05060,SSQGGSLPSEEK,4.676500e+01
399116,49239_12,12,49239,P02774,RTHLPEVFLSK,4.885360e+01
912621,49239_60,60,49239,P02774,RTHLPEVFLSK,4.937430e+01
...,...,...,...,...,...,...
746047,7508_48,48,7508,P02768,KVPQVSTPTLVEVSR,1.316000e+08
589502,59574_24,24,59574,P02768,KVPQVSTPTLVEVSR,1.317480e+08
183119,7117_6,6,7117,P02768,KVPQVSTPTLVEVSR,1.358410e+08
418679,56691_12,12,56691,P02768,KVPQVSTPTLVEVSR,1.443830e+08


In [15]:

peptides_data_protein=pd.pivot_table(peptides_data, values='PeptideAbundance', index=["visit_id","visit_month"],
                    columns="UniProt", aggfunc=[np.mean])

peptides_data_protein.head()



mean                                             \
UniProt                 O00391        O00533   O00584   O14498     O14773   
visit_id visit_month                                                        
10053_0  0             9104.27  57474.485714      NaN      NaN   7150.570   
10053_12 12           10464.20  62226.528571      NaN      NaN        NaN   
10053_18 18           13235.70  72483.685714  7126.96  24525.7        NaN   
10138_12 12           12600.20  61822.562500  9165.06  27193.5  11253.045   
10138_24 24           12003.20  65267.162500  4498.51  17189.8  14556.200   

                                                                           \
UniProt                O14791        O15240    O15394    O43505    O60888   
visit_id visit_month                                                        
10053_0  0            2497.84  41501.450000  15113.60  167327.0  129048.0   
10053_12 12               NaN  98558.300000  15099.10  164268.0  108114.0   
10053_18 18           2372.71  42168.533333  16289.60  168107.0  163776.0   
10138_12 12           6015.90  52104.496667  27273.20  204013.0   56725.0   
10138_24 24           2665.15  50389.850333  26169.05  240892.0   85767.1   

                      ...                                                    \
UniProt               ...    Q9HDC9    Q9NQ79    Q9NYU2   Q9UBR2     Q9UBX5   
visit_id visit_month  ...                                                     
10053_0  0            ...       NaN   9469.45   94237.6      NaN  23016.000   
10053_12 12           ...       NaN  14408.40       NaN      NaN  28537.000   
10053_18 18           ...  317477.0  19333.60  111107.0      NaN  18966.330   
10138_12 12           ...  557904.0  22278.45  155619.0  14647.9  18463.845   
10138_24 24           ...       NaN  23918.40  177619.0  17061.1  12755.200   

                                                                         
UniProt                   Q9UHG2    Q9UKV8   Q9UNU6    Q9Y646    Q9Y6R7  
visit_id visit_month                                                     
10053_0  0            88991.6500   65900.0  15382.0       NaN  19017.40  
10053_12 12           85866.7000   65668.1      NaN   9295.65  25697.80  
10053_18 18           61296.9900   59986.1  10813.3       NaN  14551.35  
10138_12 12           57307.8275  106564.0  26077.7  21441.80   7642.42  
10138_24 24           44180.5975   59471.4  12639.2  15091.40   6168.55  

[5 rows x 227 columns]

In [29]:
peptides_data_protein.isna().sum()

      UniProt
mean  O00391     349
      O00533       1
      O00584      13
      O14498      81
      O14773      66
                ... 
      Q9UHG2       0
      Q9UKV8     206
      Q9UNU6     352
      Q9Y646      61
      Q9Y6R7      69
Length: 227, dtype: int64

In [36]:
SD_list = peptides_data_protein.describe()
SD_list.head()

mean                                                          \
UniProt        O00391        O00533        O00584        O14498        O14773   
count      764.000000   1112.000000   1100.000000   1032.000000   1047.000000   
mean     11641.264435  67247.054945  13994.004630  27305.934884  11072.792424   
std       2817.003530  28664.669538   5104.622703   8446.187506   4285.765550   
min        873.778000  14929.560000    591.103000   8945.340000   2255.075000   
25%       9736.857500  47803.891071  10589.583750  21123.650000   8456.857500   

                                                                 \
UniProt       O14791        O15240        O15394         O43505   
count     942.000000   1113.000000   1050.000000    1110.000000   
mean     3004.990691  49259.308752  28590.681643  195599.363694   
std      1142.159575  24830.506031  10809.893658   79739.704279   
min       336.517000   8056.869000   3306.880000   29740.900000   
25%      2189.087500  31953.013333  20990.875000  142054.500000   

                        ...                                              \
UniProt         O60888  ...         Q9HDC9        Q9NQ79         Q9NYU2   
count      1079.000000  ...     959.000000   1075.000000     957.000000   
mean     145382.047368  ...  280939.076194  19156.512028  111896.595298   
std       58028.089713  ...  133832.119821   8443.939637   24157.050158   
min        8358.080000  ...    9844.570000   4704.100000   47111.700000   
25%      103983.000000  ...  196609.000000  13106.130000   94273.600000   

                                                                               \
UniProt        Q9UBR2        Q9UBX5        Q9UHG2        Q9UKV8        Q9UNU6   
count      972.000000   1100.000000   1113.000000    907.000000    761.000000   
mean     15228.332809  18985.308521  64073.870634  69924.939658  19742.853219   
std       4564.470310   7684.025913  30537.293247  25943.573085   6551.077082   
min       4469.080000   1571.400000   8137.330000   3439.810000   3950.620000   
25%      12071.450000  13998.526250  43932.657500  54132.600000  15039.900000   

                                     
UniProt        Q9Y646        Q9Y6R7  
count     1052.000000   1044.000000  
mean     19884.525447  13095.070340  
std       5941.115219   5359.745923  
min       2038.500000   2454.385000  
25%      15785.775000   9231.288750  

[5 rows x 227 columns]

In [38]:
new_row = SD_list.iloc[2] / SD_list.iloc[1]
new_row.name = 'SEM'
SD_list = SD_list.append(new_row)

      UniProt
mean  P01008     0.183777
      P06681     0.186507
      P08697     0.195364
      P25311     0.196763
      P02760     0.196861
                   ...   
      P01877     0.668048
      P01876     0.679917
      P01861     0.745871
      P00738     0.817048
      P16152     1.177408
Name: SEM, Length: 227, dtype: float64